# Importing Data - WIKI
- Scraping names and committee names of Senate Committee Members
- Scraping individual data from wikipedia from the previous list created
- Creating dataframes of both committee memebers ranking member and chair 
- Further work will include voter data, state, assumed office, time in office

In [1]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime

In [2]:
# Run this code to download/scrape the data off of wikipedia page
# url = "https://en.wikipedia.org/wiki/List_of_United_States_Senate_committees"
# response = requests.get(url)
# soup = bs(response.text, 'html.parser')
# #print(soup.prettify())
# names = pd.read_html(url)
# sliceNames = names[0]
# #Exportng data from Webscraping to CSV
# sliceNames.to_csv('names.csv', index = False)

In [3]:
#Importing Webscraping from CSV
importNames = pd.read_csv('names.csv')

In [4]:
committee = importNames['Committee']
chairNames = importNames['Chair']
rankingMembers = importNames['Ranking Member']

In [5]:
chairNames = list(chairNames)
chairNames = chairNames[1:]
rankingMembers = rankingMembers[1:]

In [6]:
#Pulling out list of names from scrape
#chairNames[1][:12]
def stripNames(stripList):
    stateParty = []
    fullName = []
    party = []
    state = []
    for i in stripList:
        counter = 0
        for j in i:
            counter += 1
            if j == '(':
                stateParty.append(i[counter-1:])
                fullName.append(i[:counter-2])
                party.append(i[-5:len(i)-4])
                state.append(i[-3:len(i)-1])
                #print(counter)
    return fullName, stateParty, party, state

In [7]:
chairNamesCols = stripNames(chairNames)
rankingMembersCols = stripNames(rankingMembers)
len(importNames)
importNames = importNames.drop(importNames.index[0])
#print(len(importNames))

In [8]:
#Renaming columns to normalize data
importNames['Chair_Names'] = chairNamesCols[0]
importNames['Party'] = chairNamesCols[1]
importNames['PartyAbbv'] = chairNamesCols[2]
importNames['State'] = chairNamesCols[3]
importNames['Ranking_Members'] = rankingMembersCols[0]
importNames['Party'] = rankingMembersCols[1]
importNames['PartyAbbv'] = rankingMembersCols[2]
importNames['State'] = rankingMembersCols[3]

In [9]:
importNames.head(2)

,Committee,Committee.1,Chair,Ranking Member,Refs,Chair_Names,Party,PartyAbbv,State,Ranking_Members
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts (R-KS),Debbie Stabenow (D-MI),[1][2],Pat Roberts,(D-MI),D,MI,Debbie Stabenow
2,NaN,"Commodities, Risk Management and Trade",John Boozman (R-AR),Sherrod Brown (D-OH),[3],John Boozman,(D-OH),D,OH,Sherrod Brown


In [10]:
#Renaming and dropping columns to normalize data
importNames['Committee'] = importNames['Committee'].replace(np.NaN, 'SubCommittee')
importNames = importNames.drop(columns=['Refs', 'Ranking Member', 'Chair', 'Party'])
importNames = importNames.rename(columns={'Committee.1':'SubCommittee', 'PartyAbbv':'RM_Party', 'State':'State_Abbrv'})

In [11]:
importNames

,Committee,SubCommittee,Chair_Names,RM_Party,State_Abbrv,Ranking_Members
1,"Agriculture, Nutrition and Forestry","Agriculture, Nutrition and Forestry",Pat Roberts,D,MI,Debbie Stabenow
2,SubCommittee,"Commodities, Risk Management and Trade",John Boozman,D,OH,Sherrod Brown
3,SubCommittee,"Conservation, Forestry and Natural Resources",Mike Braun,D,CO,Michael Bennet
4,SubCommittee,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith,D,NY,Kirsten Gillibrand
5,SubCommittee,"Nutrition, Agricultural Research and Specialty...",Deb Fischer,D,PA,Bob Casey
...,...,...,...,...,...,...
80,SubCommittee,Intellectual Property,Thom Tillis,D,DE,Chris Coons
81,SubCommittee,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse,D,CT,Richard Blumenthal
82,Rules and Administration,Rules and Administration,Roy Blunt,D,MN,Amy Klobuchar
83,Small Business and Entrepreneurship,Small Business and Entrepreneurship,Marco Rubio,D,MD,Ben Cardin


In [12]:
#Creating a column Committee which has if member is in a subCommittee or main Committee
replaceCommittee = []
for i in importNames['Committee']:
    if i == 'SubCommittee':
        replaceCommittee.append(i)
        continue
    else:
        replaceCommittee.append('Committee')
importNames = importNames.drop(columns=['Committee'])
importNames['Committee'] = replaceCommittee

In [13]:
importNames.tail()

,SubCommittee,Chair_Names,RM_Party,State_Abbrv,Ranking_Members,Committee
80,Intellectual Property,Thom Tillis,D,DE,Chris Coons,SubCommittee
81,"Oversight, Agency Action, Federal Rights and F...",Ben Sasse,D,CT,Richard Blumenthal,SubCommittee
82,Rules and Administration,Roy Blunt,D,MN,Amy Klobuchar,Committee
83,Small Business and Entrepreneurship,Marco Rubio,D,MD,Ben Cardin,Committee
84,Veterans' Affairs,Jerry Moran,D,MT,Jon Tester,Committee


In [14]:
#Creating a column for full state name for visualizations as there are only abbreviations in the original data
stateName = pd.read_csv('stateAbbrv.csv')
stateDict = dict(zip(stateName['Code'], stateName['State']))
def longState(search_value):
    for key, val in stateDict.items():
        if key == search_value:
            return val
fullStateName = []
for i in importNames['State_Abbrv']:
    fullStateName.append(longState(i))
importNames['State'] = fullStateName

In [15]:
#Creating Columns for both Parties of Chair Members and Ranking Members
#The Chair Members are always the member of the majority party
rankingParty = []
chairParty = []
chairPartyAbbv = []
for i in importNames['RM_Party']:
    rankingParty.append('Democrat')
    chairParty.append('Republican')
    chairPartyAbbv.append('R')
importNames['R_Party_Name'] = rankingParty
importNames['C_Party_Name'] = chairParty
importNames['C_Party'] = chairPartyAbbv

In [16]:
importNames.head()

,SubCommittee,Chair_Names,RM_Party,State_Abbrv,Ranking_Members,Committee,State,R_Party_Name,C_Party_Name,C_Party
1,"Agriculture, Nutrition and Forestry",Pat Roberts,D,MI,Debbie Stabenow,Committee,Michigan,Democrat,Republican,R
2,"Commodities, Risk Management and Trade",John Boozman,D,OH,Sherrod Brown,SubCommittee,Ohio,Democrat,Republican,R
3,"Conservation, Forestry and Natural Resources",Mike Braun,D,CO,Michael Bennet,SubCommittee,Colorado,Democrat,Republican,R
4,"Livestock, Marketing and Agriculture Security",Cindy Hyde-Smith,D,NY,Kirsten Gillibrand,SubCommittee,New York,Democrat,Republican,R
5,"Nutrition, Agricultural Research and Specialty...",Deb Fischer,D,PA,Bob Casey,SubCommittee,Pennsylvania,Democrat,Republican,R


In [17]:
#Locating Names with a Middle Name in Chair Names for Renaming
# i = 1
# while i < len(importNames['Chair_Names']):
#     printHolder = importNames["Chair_Names"][i]
#     #print(printHolder)
#     if 'Shelley' in printHolder:
#         #print(f'{printHolder}, {i}')
#         i += 1
#     elif 'VACANT' in printHolder:
#         #print(f'{printHolder}, {i}')
#         i += 1
#     else:
#         i += 1

In [18]:
#Locating Names with a Middle Name in Ranking Members for Renaming
# i = 1
# while i < len(importNames['Ranking_Members']):
#     printHolder = importNames["Ranking_Members"][i]
#     #print(printHolder)
#     if 'Catherine' in printHolder:
#         print(f'{printHolder}, {i}')
#         i += 1
#     elif 'VACANT' in printHolder:
#         print(f'{printHolder}, {i}')
#     elif 'Van' in printHolder:
#         print(f'{printHolder}, {i}')
#         i += 1
#     else:
#         i += 1

In [19]:
#Renaming columns to better split names 
#Ideally there would be a third column with middle name or middle initial if that were available
#However, there would be alot of NaN's and the information would have to come from elsewhere
importNames["Chair_Names"][13] = "ShelleyMoore Capito"
importNames["Chair_Names"][51] = "ShelleyMoore Capito"
importNames["Chair_Names"][65] = "VACANT VACANT"
importNames['Ranking_Members'][29] = 'CatherineCortez Masto'
importNames['Ranking_Members'][46] = 'CatherineCortez Masto'
importNames['Ranking_Members'][33] = 'ChrisVan Hollen'

In [20]:
#importNames.tail(60)

In [21]:
def splitName(column):
    splitName = []
    for i in column:
        splitName.append(i.split())
    Namelist = []
    noMiddle =[]
    for i in column:
        Namelist.append(i.split(' '))
    len(Namelist)
    noMiddle = []
    counter = 1
    for i in Namelist:
        for j in i:
            if len(j) > 1:
                noMiddle.append(j)
                counter += 1
            elif len(j) <= 2:
                print(f'{j} <= len(j)')
                noMiddle.append(j)
                counter += 1
                #continue
    #             query = input('Keep this value, Y/N?')
    #             if query == 'Y':
    #                 noMiddle.append(j)
    #                 counter += 1
    #             else:
    #                 continue
    #                 counter += 1
            else:
                print(f'{j}')
                counter += 1
    #print(len(noMiddle))
    firstName = noMiddle[:len(noMiddle)+1:2]
    lastName = noMiddle[1:len(noMiddle)+1:2]
    #print(len(lastName))
    #print(len(firstName))
    #print(len(column))
    return firstName, lastName

In [22]:
chairNamesSplit = splitName(importNames["Chair_Names"])
rankingNamesSplit = splitName(importNames['Ranking_Members'])

In [23]:
importNames['C_First_Name'] = chairNamesSplit[0]
importNames['C_Last_Name'] = chairNamesSplit[1]
importNames['R_First_Name'] = rankingNamesSplit[0]
importNames['R_Last_Name'] = rankingNamesSplit[1]

In [24]:
#importNames.head(2)

In [25]:
importNames = importNames[['Committee','SubCommittee', 'C_First_Name', 'C_Last_Name', 'C_Party', 'C_Party_Name', 'State_Abbrv', 'R_First_Name', 'R_Last_Name', 'RM_Party', 'R_Party_Name', 'State', 'Chair_Names', 'Ranking_Members']]

In [26]:
importNames.head(2)

,Committee,SubCommittee,C_First_Name,C_Last_Name,C_Party,C_Party_Name,State_Abbrv,R_First_Name,R_Last_Name,RM_Party,R_Party_Name,State,Chair_Names,Ranking_Members
1,Committee,"Agriculture, Nutrition and Forestry",Pat,Roberts,R,Republican,MI,Debbie,Stabenow,D,Democrat,Michigan,Pat Roberts,Debbie Stabenow
2,SubCommittee,"Commodities, Risk Management and Trade",John,Boozman,R,Republican,OH,Sherrod,Brown,D,Democrat,Ohio,John Boozman,Sherrod Brown


In [27]:
rankingMembers = importNames[['Committee', 'SubCommittee', 'R_First_Name', 'R_Last_Name', 'RM_Party', 'State_Abbrv', 'Ranking_Members', 'State', 'R_Party_Name']]
chairNames = importNames[['Committee','SubCommittee', 'C_First_Name', 'C_Last_Name', 'C_Party', 'State_Abbrv', 'Chair_Names', 'State', "C_Party_Name"]]

In [28]:
#chairNames.head()

In [29]:
importNames.to_csv('Chairs_RankingMembers.csv', index = False)
rankingMembers.to_csv('Ranking_Members.csv', index = False)
chairNames.to_csv('Chair_Members.csv', index = False)

In [30]:
chairNames[:50]

,Committee,SubCommittee,C_First_Name,C_Last_Name,C_Party,State_Abbrv,Chair_Names,State,C_Party_Name
1,Committee,"Agriculture, Nutrition and Forestry",Pat,Roberts,R,MI,Pat Roberts,Michigan,Republican
2,SubCommittee,"Commodities, Risk Management and Trade",John,Boozman,R,OH,John Boozman,Ohio,Republican
3,SubCommittee,"Conservation, Forestry and Natural Resources",Mike,Braun,R,CO,Mike Braun,Colorado,Republican
4,SubCommittee,"Livestock, Marketing and Agriculture Security",Cindy,Hyde-Smith,R,NY,Cindy Hyde-Smith,New York,Republican
5,SubCommittee,"Nutrition, Agricultural Research and Specialty...",Deb,Fischer,R,PA,Deb Fischer,Pennsylvania,Republican
6,SubCommittee,Rural Development and Energy,Joni,Ernst,R,MN,Joni Ernst,Minnesota,Republican
7,Committee,Appropriations,Richard,Shelby,R,VT,Richard Shelby,Vermont,Republican
8,SubCommittee,"Agriculture, Rural Development, Food and Drug ...",John,Hoeven,R,OR,John Hoeven,Oregon,Republican
9,SubCommittee,"Commerce, Justice, Science and Related Agencies",Jerry,Moran,R,NH,Jerry Moran,New Hampshire,Republican
10,SubCommittee,Defense,Richard,Shelby,R,IL,Richard Shelby,Illinois,Republican
